Only run if in Colab!

In [ ]:
!git clone https://github.com/ratinac-nikola/rbm.git
%cd rbm
%pip install -r requirements.txt

In [ ]:
%pwd

### Initialization

In [ ]:
from sklearn.model_selection import train_test_split
import torch
import config

if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device('cuda')
else:
    print("Using CPU")
    device = "cpu"

In [ ]:
from dataset import Dataset
dataset = Dataset(device=device)
tensorData = dataset.getDatasetAsTensor()
trainset, testset = train_test_split(tensorData, train_size=0.8)
trainset.shape, testset.shape

In [ ]:
from rbm import RBM
rbm = RBM(tensorData.shape[1], n_hidden=10, device=device)

In [ ]:
from utils import *
for epoch in range(1, config.epochs + 1):

    for vin in trainset:

        v0 = vin
        vk = vin

        for k in range(10):
            _, hk = rbm.sample_h(vk)  # forward pass
            _, vk = rbm.sample_v(hk)  # backward pass
            vk[v0.sum(dim=1) == 0] = v0[v0.sum(dim=1) == 0]

        rbm.train(v0, vk)

    print(f"Epoch {epoch} done.", end=" ")
    rmse = 0
    for vin in testset:
        rec = rbm.reconstruct(vin)
        # print(reconstruction_rmse(vin, rec))
        rmse += reconstruction_rmse(vin, rec)
    print("RMSE:",rmse / len(testset))